In [30]:
import torch;
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import WeightedRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
from tqdm import tqdm
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from dataloader import *
from cnn_model_1 import *

In [2]:
#only on validation set
csv_path = 'progress.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

CDL = CLUnivDataLoader(csv_path, img_path, transform = transforms.Compose([transforms.Resize(64),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),timestep=3,n=2)

#random sampler
batch_size = 64
validation_split = .2
test_split = .2
train_split = 1-validation_split-test_split
shuffle_dataset = True

# Creating data indices for training and validation splits:
dataset_size = len(CDL)
indices = list(range(dataset_size))
split_train = int(np.floor(train_split * dataset_size))
split_valid = int(np.floor((train_split+validation_split) * dataset_size))
if shuffle_dataset :
    np.random.shuffle(indices)
train_indices, val_indices, test_indices =  indices[:split_train],indices[split_train:split_valid],indices[split_valid:]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(CDL, batch_size=batch_size, sampler=test_sampler)

In [18]:
targets = np.array(CDL.gettargets())

#


In [22]:
targets[targets>-2] = 1
targets[targets < -2] = 0

In [25]:
class_sample_count = np.array([len(np.where(targets == t)[0]) for t in np.unique(targets)])


In [28]:
weight = 1. / class_sample_count
samples_weight = np.array([weight[int(t)] for t in targets])

In [30]:
wrs = WeightedRandomSampler(samples_weight, len(samples_weight))
train_loader = torch.utils.data.DataLoader(trainsd, batch_size=batch_size, sampler=train_sampler)

NameError: name 'WeightedRandomSampler' is not defined

In [10]:
def gen_weightedsampler(datasets):
    target = datasets.labels
    class_sample_count = np.array(
        [len(np.where(target == t)[0]) for t in np.unique(target)])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in target])
    samples_weight = torch.from_numpy(samples_weight)
    return WeightedRandomSampler(samples_weight, len(samples_weight))
    

['AL0320_2005_02_16_OS.JPG', 'AL0320_2006_01_13_OS.JPG', 'AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG']
['AL0320_2006_01_13_OS.JPG', 'AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG', 'AL0320_2014_02_18_OS_02.JPG']
['AL0320_2007_03_26_OS.JPG', 'AL0320_2011_04_26_OS_02.JPG', 'AL0320_2012_07_25_OS_02.JPG', 'AL0320_2014_02_18_OS_02.JPG', 'AL0320_2015_03_19_OS_02.JPG']
['AL0320_2005_02_16_OD.JPG', 'AL0320_2006_01_13_OD.JPG', 'AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG']
['AL0320_2006_01_13_OD.JPG', 'AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG', 'AL0320_2014_02_18_OD_02.JPG']
['AL0320_2007_03_26_OD.JPG', 'AL0320_2011_04_26_OD_01.JPG', 'AL0320_2012_07_25_OD_02.JPG', 'AL0320_2014_02_18_OD_02.JPG', 'AL0320_2015_03_19_OD_01.JPG']
['AL0348_2004_05_05_OS.JPG', 'AL0348_2005_04_26_OS.JPG', 'AL0348_2007_04_30_OS.JPG', 'AL0348_2011_05

In [1]:
import pandas as pd
import torch
from torch.utils.data.dataset import Dataset
from torchvision import transforms, utils
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import PIL
import os

In [3]:
dataset = pd.read_csv('pairwise_lstm.csv')

In [6]:
grouped = dataset.groupby('maskedeye')
grouped = iter(grouped)
_, chunk = next(grouped)
chunks = []
targets = []
seqlen = []


In [28]:
_, chunk = next(grouped)
chunk

,filename,maskedeye,pdate,examdate_shift,rnflmean_g,days,target
15,AL0349_2011_06_02_OD_01.JPG,al0349r,2011-06-02,2011-06-02,105.66666,0,-0.616978
16,AL0349_2012_04_12_OD_02.JPG,al0349r,2012-04-12,2012-04-12,105.00000,315,-0.747498


In [26]:

for i in range(1,chunk.shape[0]):
    chunks += [[chunk['filename'].iloc[0],chunk['filename'].iloc[i]]]
    targets += [chunk['target'].iloc[i]]

In [27]:
targets

[-0.7351501,
 -0.28899708,
 -0.81268001,
 -0.1204906,
 -0.70073372,
 -0.69243568,
 -0.89236963,
 -1.0637892,
 -0.71282333]

In [44]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [45]:
get_n_params(model)

19092097

In [32]:
im1 = torch.rand((3,64,64))
im2 = torch.rand((3,64,64))
x = torch.stack([im1,im2],0)

In [37]:
x = torch.unsqueeze(x, 0)

In [39]:
ts = 2
model = CNNLSTMNet(ts=ts)

In [41]:
o = model(x)

In [42]:
o.shape

torch.Size([1, 1])

In [148]:
o = model1(o)

In [149]:
o

tensor([[-0.0596, -0.0491]], grad_fn=<AddmmBackward>)

In [134]:
1024/4

256.0